# Traffic sign classifier neural network
```Trunk code by Vivek Kumar, commentary and attempts to develop the LeCun et al. solution by M.L.```

### Goals of this exercise
* Reproduce LeCun & team classification results using Python ML-libraries
* Improve if possible by trying the following
    * Standardize luminosity and colours (LeCun et al. ended up dropping colours, which I see as a fail, human perceptual system normalizes light and colour hues, how about trying the same and retaining colour as a classifying input? It is pretty solid classifier - it seems frankly stupid to let the neural network be misguided due to seeing a yellow in the dark as different colour than yellow in the light? I think we could attempt to standardize colours so that the machine sees "a yellow" and "a red" etc. and try what kind of predictions we end up with.)
    * adding jitter was also suggested



In [ ]:
# we need this for handling serialized binary objects 
# (which the pics are apparently)
import pickle

# OK, but which sets are the ones you used and on what basis did you select?
# there was quite a plethora of different datasets on offer
# haar, hog, gt, hue-hist? Wut?
# Did you do pre-processing other than what is seen here, the "multilabel binarizer"? What does it do?
# If yes, what exactly?
     # 100K  GTSRB_Final_Test_GT.zip
     # 305M  GTSRB_Final_Test_Haar.zip
     # 279M  GTSRB_Final_Test_HOG.zip
     # 4.6M  GTSRB_Final_Test_HueHist.zip
     #  85M  GTSRB_Final_Test_Images.zip
     # 945M  GTSRB_Final_Training_Haar.zip
     # 864M  GTSRB_Final_Training_HOG.zip
     #  18M  GTSRB_Final_Training_HueHist.zip
     # 264M  GTSRB_Final_Training_Images.zip

training_file = "/Users/Masters/Downloads/traffic-signs-data/train.p"
validation_file= "/Users/Masters/Downloads/traffic-signs-data/valid.p"
testing_file = "/Users/Masters/Downloads/traffic-signs-data/test.p"

# loading in the image populations
# ..have you had binary file corruption on Windows with this "read binary" -mode
with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)

# labels. This confounds me. Since X is the input and y is the output class
# is "label" just a name for the output class?
#...if that, where did they come from, the labels, i.e. the target classes?
# I need to check the schema/shape of the pickle
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']


In [ ]:
import numpy as np

# population samples to vars with len
n_train = len(X_train)
n_validation = len(X_valid)
n_test = len(X_test)

image_shape = X_train.shape[1:]

# what do you do here and why?
classes = np.append(y_train, y_valid)
classes = np.append(classes, y_test)
n_classes = len(set(classes))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

In [ ]:
# Print histograms of training target distribution with 43 bins
hist, bins = np.histogram(y_train, bins=43)
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
plt.show()

In [ ]:
# what do you do here and why?
X_train = np.sum(X_train/3, axis=3, keepdims=True)
X_valid = np.sum(X_valid/3, axis=3, keepdims=True)
X_test = np.sum(X_test/3, axis=3, keepdims=True)

X_train = (X_train - 128)/128
X_valid = (X_valid - 128)/128
X_test = (X_test - 128)/128

In [ ]:
# bunch of different CNN layer types here: convolution, max-pooling
# Rectified linear units (relu) chosen to enhance non-linear properties of network 
# (but isn't this a layer type in itself, it is in "activation"
# What do strides do in NN usage?
# input_shape 32,32,1 ...32 px * 32 px and..1?


from keras.models import Model
from keras.layers import Input
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Concatenate
from keras.layers.convolutional import AtrousConvolution2D
from keras import backend as K


def model_traffic_sign(input_tensor=None):
    input_shape = (32, 32, 1)

    img_input = Input(shape=input_shape)
    
# how does this layering work - you just re-assign x...and then build another layer, 
# at the end of which you kinda concatenate the previous layer?
# Which documentation part should I read here? - keras.layers -api?

    # conv_1
    x = Convolution2D(6, (5, 5), activation='relu', padding='valid', name='block1_conv1')(img_input)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # conv_2
    x = Convolution2D(16, (5, 5), activation='relu', padding='valid', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
    
    layer2 = x
    
    x = Convolution2D(400, (5, 5), activation='relu', padding='valid', name='block2_conv1')(x)
    #x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
    
    x = Flatten()(x)
    layer2_flat = Flatten()(layer2)
    
    x = Concatenate()([x, layer2_flat])
    #x = Dense(120, activation="relu")(x)
    x = Dense(43, activation="softmax")(x)

    
    model = Model(img_input, x)

    return model

In [ ]:
# stochastic gradient descent optimization algo variant
from keras.optimizers import Adam 

model = model_traffic_sign()
optim = Adam(lr = 0.0009) # what is this magic number and where is it from?
model.compile(optim, loss='categorical_crossentropy')
model.summary()

In [ ]:
from tensorflow.keras.utils import Sequence
class MySequence(Sequence):

        def __init__(self, x_set, y_set, batch_size):
            self.x, self.y = x_set, y_set
            self.batch_size = batch_size

        def __len__(self):
            return math.ceil(len(self.x) / self.batch_size)

        def __getitem__(self, idx):
            batch_x = self.x[idx * self.batch_size:(idx + 1) *
            self.batch_size]
            batch_y = self.y[idx * self.batch_size:(idx + 1) *
            self.batch_size]

            return batch_x,  batch_y

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

classes = classes.reshape(len(classes),1)
lbl_enc = MultiLabelBinarizer()
lbl_enc.fit(classes)
lbs_train = lbl_enc.transform(y_train.reshape(len(y_train), 1))
lbs_valid = lbl_enc.transform(y_valid.reshape(len(y_valid), 1))
lbs_test = lbl_enc.transform(y_test.reshape(len(y_test), 1))

In [ ]:
from sklearn.utils import shuffle


n_epoches = 60
batch_size = 256
n_steps = int(n_train / batch_size)

for i in range(n_epoches):
    X_train, lbs_train = shuffle(X_train, lbs_train)
    data_gen = MySequence(X_train, lbs_train, batch_size=batch_size)

    for j in range(n_steps):
        x, y = data_gen[j]
        loss = model.train_on_batch(x, y)
        
    val_pred = model.predict(X_valid)
    pred = np.argmax(val_pred, 1)
    true = np.where(lbs_valid == 1)[1]
    correct = sum(np.equal(true, pred))
    val_acc = correct/len(X_valid)
    print("loss: {}".format(loss))
    print("epoch no-> {} val_acc-> {}".format(i, val_acc))
        

In [ ]:
test_pred = model.predict(X_test)
pred = np.argmax(test_pred, 1)
true = np.where(lbs_test == 1)[1]
correct = sum(np.equal(true, pred))
test_acc = correct/len(X_test)

In [ ]:
train_pred = model.predict(X_train)
pred = np.argmax(train_pred, 1)
true = np.where(lbs_train == 1)[1]
correct = sum(np.equal(true, pred))
train_acc = correct/len(X_train)